In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from math import cos, acos, sin , asin, pi, radians, sqrt, degrees, atan2
import math

In [ ]:
##  Data Processing

def dataProcess(inputFile):
  tempData = []
  with open(inputFile) as f:
    for line in f:
      tempString = line
      dateString = tempString[0:9]
      dateString = dateString.replace(" ", ":")
      finalString = dateString + tempString[9:len(tempString)]
      finalString = finalString.strip("\n")
      finalList = finalString.split(" ")
      tempData.append(finalList)

  data = pd.DataFrame(tempData, columns = ["Date", "Time", "RA", "Dec", "Sun Vector X", "Sun Vector Y", "Sun Vector Z"])
  data['RA'] = raToDecimal(data['RA'])
  data['Dec'] = decToDecimal(data['Dec'])

  data['Sun Vector X'] = pd.to_numeric(data['Sun Vector X'])
  data['Sun Vector Y'] = pd.to_numeric(data['Sun Vector Y'])
  data['Sun Vector Z'] = pd.to_numeric(data['Sun Vector Z'])
  data['JD'] = obtainJDDates(data['Date'], data['Time'])
  return data


def raToDecimal(data):
  decimalRA = []
  for i in range(len(data)):
    tempValues = np.array(data[i].split(":")).astype(float)
    tempDecimal = (tempValues[0] + (tempValues[1]/60) + (tempValues[2]/3600)) * 15
    tempDecimal = radians(tempDecimal)
    decimalRA.append(tempDecimal)
  return decimalRA


def decToDecimal(data):
  decimalDEC = []
  for i in range(len(data)):
    tempString = data[i]
    tempValues = np.array(tempString.split(":")).astype(float)
    if tempValues[0] >= 0:
      tempDecimal = tempValues[0] + (tempValues[1]/60) + (tempValues[2]/3600)
    else:
      tempDecimal = tempValues[0] - (tempValues[1]/60) - (tempValues[2]/3600)
    tempDecimal = radians(tempDecimal)
    decimalDEC.append(tempDecimal)
  return decimalDEC


def obtainJDDates(dateData, timeData):
  dataJD = []
  for i in range(len(dateData)):
    tempDateData = np.array(dateData[i].split(":")).astype(int).tolist()
    tempTimeData = np.array(timeData[i].split(":")).tolist()
    
    H = int(tempTimeData[0])
    M = int(tempTimeData[1])
    S = int(float(tempTimeData[2]))

    tempTime = pd.Timestamp(year = tempDateData[0], month = tempDateData[1], day = tempDateData[2], hour = H, minute = M, second = S)
    tempTime = tempTime.to_julian_date()
    dataJD.append(float(tempTime))

  return np.array(dataJD)

In [ ]:
def obtainRowHats(raData, decData):

  iHat = np.array([1, 0, 0])
  jHat = np.array([0, 1, 0])
  kHat = np.array([0, 0, 1])

  rowHats = []

  for i in range(len(raData)):
    tempPVec = (cos(raData[i]) * cos(decData[i])) * iHat + (sin(raData[i]) * cos(decData[i])) * jHat + sin(decData[i]) * kHat
    rowHats.append(tempPVec)
  
  return rowHats

In [ ]:
def obtainDValues(rowHats, sunVectors):
  
  dValues = []

  for i in range(3):

    d1 = np.dot(np.cross(sunVectors[i], rowHats[1]), rowHats[2])
    d2 = np.dot(np.cross(rowHats[0], sunVectors[i]), rowHats[2])
    d3 = np.dot(np.cross(rowHats[1], sunVectors[i]), rowHats[0])

    dValues.append(d1)
    dValues.append(d2)
    dValues.append(d3)

  d0 = np.dot(np.cross(rowHats[1], rowHats[2]), rowHats[0])

  return d0, dValues[0], dValues[1], dValues[2], dValues[3], dValues[4], dValues[5], dValues[6], dValues[7], dValues[8]

In [ ]:
def obtainMagnitude(vector):
  vectorMag = math.sqrt(vector[0]**2 + vector[1]**2 + vector[2]**2)
  return vectorMag

In [ ]:
#Scalar Equation of Lagrange

def SEL(taus, Sun2, rhohat, Dvalues, choose = True):
  taus = np.array(taus)
  Sun2 = np.array(Sun2)
  rhohat = np.array(rhohat)

  mu = 1
  t = taus[2]
  a1 = taus[1]/t 
  b1 = (a1/6)*(t**2 - taus[1]**2)
  a3 = (-taus[0])/t
  b3 = ((a3)/6) * (t**2 - taus[0]**2)

  A = (a1 * Dvalues[1] - Dvalues[2] + a3 * Dvalues[3])/(- Dvalues[0])
  B = (b1 * Dvalues[1] + b3 * Dvalues[3])/( - Dvalues[0])

  F = obtainMagnitude(Sun2)**2
  E = -2 * (np.dot(rhohat,Sun2))
  
  aCoeff = - (A**2 + A * E + F)
  bCoeff = - mu * (2 * A * B + B * E)
  cCoeff = - (mu ** 2) * (B ** 2)

  coEffArray = np.asarray([1, 0, aCoeff, 0, 0, bCoeff, 0, 0, cCoeff])

  roots = np.roots(coEffArray)

  #Obtain only real positive numbers
  for i in range(len(roots)):
    if ((np.isreal(roots[i]) == False)):
      roots[i] = 0

  roots = roots[roots != 0]
  roots = roots[roots >= 0]
  roots = roots.astype(float)

  #Obtain Rhos

  rows = []
  for i in range(len(roots)):
    rows.append(A + (B/((roots[i])**3)))
    
  rows = np.array(rows)

  #Filter Roots from negative Rhos / negative ranges

  for i in range(len(rows)):
    if (rows[i] < 0):
      roots[i] = 0

  roots = roots[roots != 0]
  rows = rows[rows >= 0]

  if choose == True:

    print("r2Mags: ", roots.tolist())
    print("Rows: ", rows.tolist())


    indexDesired = int(input(("Which roots/rows would you like to utilize? (0 indexed): ")))
    print("\n\n")
  
  else:

    indexDesired = 0

  return roots[indexDesired], rows[indexDesired]

In [ ]:
# F and G Calculations

def getE(M, e, tolerance):
    Eguess = M
    Mguess = Eguess - e*sin(Eguess)
    while abs(Mguess - M) > tolerance:
        Mguess = Eguess - e*sin(Eguess)
        Eguess = Eguess - (M - (Eguess - (e*sin(Eguess)))) / (e*cos(Eguess) - 1)
    return Eguess

def fg(tau1, r2, flag, r2dot = None):
  mu = 1

  #Taylor Series: 1st Order

  if (flag == "1st"):

    #For First Series, the r2 paramter is the r2 Mag, not the vector
    rPositionMag = r2

    f = 1 - (mu/ (2 * (rPositionMag**3))) * tau1 ** 2
    g = tau1 - (mu/ (6 * (rPositionMag**3))) * tau1 ** 3

  if (flag == "3rd"):
    rPositionMag = obtainMagnitude(r2)
    rVelocityMag  = obtainMagnitude(r2dot)

    u = mu/(rPositionMag**3) 
    z = (np.dot(r2, r2dot))/(rPositionMag ** 2)
    q = ((np.dot(r2dot, r2dot))/(rPositionMag ** 2)) - u

    f = 1 - .5 * (u * tau1 ** 2) + .5 * (u * z * tau1 ** 3)
    g = tau1 - (mu/(6 * (rPositionMag**3))) * tau1 ** 3

  if (flag == "4th"):
    rPositionMag = obtainMagnitude(r2)
    rVelocityMag  = obtainMagnitude(r2dot)

    u = mu/(rPositionMag**3) 
    z = (np.dot(r2, r2dot))/(rPositionMag ** 2)
    q = ((np.dot(r2dot, r2dot))/(rPositionMag ** 2)) - u
  
    f = 1 - .5 * (u * tau1 ** 2) + .5 * (u * z * tau1 ** 3) + ((1/24) * (3 * u * q - 15 * u * z ** 2 + u**2) * tau1 ** 4)
    g = tau1 - ((1/6)* u * tau1 ** 3) + (1/4)*u*z*tau1**4
  
  if (flag == "Function"):

    rPositionMag = obtainMagnitude(r2)
    rVelocityMag  = obtainMagnitude(r2dot)

    #Calculate A
    a = ((2/rPositionMag) - rVelocityMag * rVelocityMag) ** - 1

    #Calculate N
    n = 1/math.sqrt((a**3))

    x = n * tau1

    while True:

      fx = x - (1 - (rPositionMag/a)) * sin(x) + (np.dot(r2, r2dot)/(n * (a**2))) * (1 - cos(x)) - n * tau1

      fxPrime = 1 - (1 - (rPositionMag/a)) * cos(x) + (np.dot(r2, r2dot)/(n * (a**2))) * sin(x)

      xNew = x - (fx/fxPrime)

      if abs(xNew - x) < 1E-12:

        x = xNew

        break

      x = xNew 

    deltaE = x

    f = 1 - (a/rPositionMag) * (1 - cos(deltaE))
    g = tau1 + (1/n) * (sin(deltaE) - deltaE)

  return round(f, 20), round(g, 20)

In [ ]:
def obtainTValues(jdData):
  k = 0.0172020989484
  t3 = k * (jdData[2] - jdData[1])
  t1 = k * (jdData[0] - jdData[1])
  t = k * (jdData[2] - jdData[0])
  return t, t1, t3

In [ ]:
def obtainCValues(f1, f3, g1, g3):
  c1 = (g3)/(f1 * g3 - g1 * f3)
  c2 = -1
  c3 = (-g1)/(f1* g3 - g1*f3)

  return c1, c2, c3

In [ ]:
def obtainRowMag(c1, c2, c3, d0, d11, d21, d31, d12, d22, d32, d13, d23, d33):
  p1 = (c1 * d11 + c2 * d12 + c3 * d13) / (c1 * d0)

  p2 = (c1 * d21 + c2 * d22 + c3 * d23) / (c2 * d0)

  p3 = (c1 * d31 + c2 * d32 + c3 * d33) / (c3 * d0)

  return p1, p2, p3

In [ ]:
def obtainSunVectors(sunXData, sunYData, sunZData):
  sunVectors = []
  for i in range(len(sunXData)):
    sunVectors.append([sunXData[i], sunYData[i], sunZData[i]])
  return sunVectors[0], sunVectors[1], sunVectors[2]

In [ ]:
def obtainAsteroidVectors(rowMagData, rowHatData, sunVectors):
  rVectors = []
  for i in range(len(rowMagData)):
    tempRVector = (rowMagData[i] * np.array(rowHatData[i])) - np.array(sunVectors[i])
    rVectors.append(np.array(tempRVector))
  return rVectors

In [ ]:
def calcNewTimeValues(timeValues, rowMags):
  newTime = []

  for i in range(3): 
    tempTime = (timeValues[i]) - (rowMags[i])/173.144632674240  #Accesses the prior time values
    newTime.append(float(tempTime))

  return newTime

In [ ]:
#Converging to find final Asteroid Vectors

def obtainConvAsteroidVectors(initialAsteroidPosVector, initialAsteroidVelVector, dValues, timeValues, rowMags, mainData):

  #Variable Definitions

  rowHats = obtainRowHats(mainData['RA'], mainData['Dec'])

  d0 = dValues[0]
  d11 = dValues[1]
  d21 = dValues[2]
  d31 = dValues[3]
  d12 = dValues[4]
  d22 = dValues[5]
  d32 = dValues[6]
  d13 = dValues[7]
  d23 = dValues[8]
  d33 = dValues[9]
  
  sunVector1, sunVector2, sunVector3 = obtainSunVectors(mainData['Sun Vector X'], mainData['Sun Vector Y'], mainData['Sun Vector Z'])

  row2MagData = [rowMags[1]]

  #Define New Vectors

  newAsteroidPosVector = initialAsteroidPosVector.tolist()

  newAsteroidVelVector = initialAsteroidVelVector.tolist()

  #Iterations 

  count = 1

  print("Iteration Convergance for Asteroid Vector:\n")

  fgFlag = str(input("\nWhat would you like to use for F & G values? (Options: 1st, 3rd, 4th, and Function): "))

  while (True):

    #Time Calculations

    newTimes = calcNewTimeValues(timeValues, rowMags)
    
    tau, tau1, tau3 = obtainTValues(newTimes)

    f1, g1 = fg(tau1, newAsteroidPosVector, fgFlag, r2dot = newAsteroidVelVector)

    f3, g3 = fg(tau3, newAsteroidPosVector, fgFlag, r2dot = newAsteroidVelVector)

    c1, c2, c3 = obtainCValues(f1, f3, g1, g3)

    row1Mag, row2Mag, row3Mag = obtainRowMag(c1, c2, c3, d0, d11, d21, d31, d12, d22, d32, d13, d23, d33)

    rowMags = [row1Mag, row2Mag, row3Mag]

    row2MagData.append(row2Mag)

    d1 = (-f3)/(f1 * g3 - f3 * g1)

    d3 = f1/(f1 * g3 - f3 * g1)

    rVectors = obtainAsteroidVectors([row1Mag, row2Mag, row3Mag], rowHats, [sunVector1, sunVector2, sunVector3])

    #Define New Asteroid Vectors

    newAsteroidPosVector = rVectors[1]

    newAsteroidVelVector = d1 * rVectors[0] + d3 * rVectors[2]

    count += 1

    print("Row2Mag Difference (" + str(count) + "):", row2MagData[len(row2MagData) - 1] - row2MagData[len(row2MagData) - 2])

    if (abs(row2MagData[len(row2MagData) - 1] - row2MagData[len(row2MagData) - 2]) < 1E-12):

      break

  return newAsteroidPosVector, newAsteroidVelVector, row2MagData[len(row2MagData) - 1]

In [ ]:
#Baby OD: Obtaining Orbital Elements

def findQuadrant(sine, cosine):
    if cosine > 0 and sine > 0: #1
        return asin(sine)

    if cosine < 0 and sine > 0: #2
        return acos(cosine)

    if cosine < 0 and sine < 0: #3
        return pi - asin(sine)

    if cosine > 0 and sine < 0: #4
        return 2*pi + asin(sine)

def obtainEcliptic(vector):
  transMatrix = [[1, 0, 0],
                 [0, cos(radians(23.4379)), sin(radians(23.4379))],
                 [0, -sin(radians(23.4379)), cos(radians(23.4379))]]

  transMatrix = np.array(transMatrix)
  vector = np.array(vector)

  ecliptic = transMatrix @ vector
  
  return ecliptic

def obtainOrbitalElements(rPosition, rVelocity, time):
  rPosition = obtainEcliptic(rPosition)
  rVelocity = obtainEcliptic(rVelocity)

  rPositionMag = obtainMagnitude(rPosition)
  rVelocityMag = obtainMagnitude(rVelocity)


  #Calculate A
  a = ((2/rPositionMag) - rVelocityMag * rVelocityMag) ** - 1

  #Calculate N
  n = 0.0172020989484/math.sqrt((a**3))

  #Calculate Eccentricity
  e = math.sqrt(1 - ((obtainMagnitude(np.cross(rPosition, rVelocity))**2)/a))

  #Caclulate Inclination
  h = np.cross(rPosition, rVelocity)
  zComp = h[2]
  I = math.acos(zComp/obtainMagnitude(h))

  #Calculate Ascending Node
  hMag = obtainMagnitude(h)
  ascNodeSin = h[0]/(hMag * sin(I))
  ascNodeCos = (-h[1])/(hMag * sin(I))
  ascNode = findQuadrant(ascNodeSin, ascNodeCos)

  #Calculate Perihelion

  #5.1 
  sinFW = rPosition[2]/(rPositionMag * sin(I))
  cosFW = (1/cos(ascNode)) * ((rPosition[0]/rPositionMag) + cos(I) * sinFW * sin(ascNode))
  FW = findQuadrant(sinFW, cosFW)

  #5.2
  cosF =  (1/e) * (((a * (1-e**2))/rPositionMag) - 1)
  posVelCross = np.dot(rPosition, rVelocity)
  sinF = (posVelCross/(e*rPositionMag))*sqrt(a*(1-e**2))

  f = findQuadrant(sinF, cosF)
  w = (FW - f)

  # Calculate Mean Anomaly

  E = acos((1/e) * (1 - (rPositionMag/a)))

  if ((f >= 0 and f <= pi) and (E > pi and E < 2 * pi)):

      E = 2 * pi - E

  elif ((f >= pi and f <= 2 * pi) and (E > 0 and E < pi)):

      E = 2 * pi - E

  M = E - e * sin(E)

  M0 = E - e * sin(E) + n * (2459784.75 - time)

  while (M0 > 2 * pi):

    M0 -= 2 * pi

  while (M < 0):

    M0 += 2 * pi

  #Solve for T

  T = ((-M0)/n) + time

  #Solve for Period

  P = ((2*pi)/n)/365.256363004

  return a, e, degrees(I), degrees(ascNode), (degrees(w) + 360)%360, degrees(M), degrees(M0), degrees(E), n, T, P

In [ ]:
#Main OD Code

def odCode(fileName):

  k = 0.0172020989484

  inputData = dataProcess(fileName)

  rowHats = obtainRowHats(inputData['RA'], inputData['Dec'])

  sunVector1, sunVector2, sunVector3 = obtainSunVectors(inputData['Sun Vector X'], inputData['Sun Vector Y'], inputData['Sun Vector Z'])

  d0, d11, d21, d31, d12, d22, d32, d13, d23, d33 = obtainDValues(rowHats, [sunVector1, sunVector2, sunVector3])

  timeValuesArray = inputData['JD'].tolist() 

  tau, tau1, tau3 = obtainTValues(inputData['JD'])

  r2Mag, thisISStupid = SEL([tau1, tau3, tau], sunVector2, rowHats[1], [d0, d21, d22, d23], choose = True)

  f1, g1 = fg(tau1, r2Mag, "1st")
  f3, g3 = fg(tau3, r2Mag, "1st")

  c1, c2, c3 = obtainCValues(f1, f3, g1, g3)

  row1Mag, row2Mag, row3Mag = obtainRowMag(c1, c2, c3, d0, d11, d21, d31, d12, d22, d32, d13, d23, d33)

  d1 = (-f3)/(f1 * g3 - f3 * g1)
  d3 = f1/(f1 * g3 - f3 * g1)

  rVectors = obtainAsteroidVectors([row1Mag, row2Mag, row3Mag], rowHats, [sunVector1, sunVector2, sunVector3])

  r2VelocityVector = d1 * rVectors[0] + d3 * rVectors[2]
  
  finalAsteroidPosVector, finalAsteroidVelVector, row2MagFinal = obtainConvAsteroidVectors(rVectors[1], r2VelocityVector, [d0, d11, d21, d31, d12, d22, d32, d13, d23, d33], timeValuesArray, [row1Mag, row2Mag, row3Mag], inputData)

  finalAsteroidPosVectorEcl = obtainEcliptic(finalAsteroidPosVector)
  finalAsteroidVelVectorEcl = obtainEcliptic(finalAsteroidVelVector)

  print("\n\nFINAL VALUES/VECTORS")
  print("\nFinal Asteroid Position Vector (Ecliptic): ", finalAsteroidPosVectorEcl)
  print("Magnitude: ", obtainMagnitude(finalAsteroidPosVectorEcl), " au")
  print("\nFinal Asteroid Velocity Vector (Ecliptic): ", k * finalAsteroidVelVectorEcl)
  print("Magnitude: ", obtainMagnitude(k * finalAsteroidVelVectorEcl), " au/day")
  print("\nRho2 = ", row2MagFinal, " au")

  a, e, I, ascNode, w, M, M0, E, n, T, P = obtainOrbitalElements(finalAsteroidPosVector, finalAsteroidVelVector, inputData['JD'][1])
  
  print("\n\nOrbital Elements:")
  print("a - Semimajor axis - (au): " + str(a))
  print("e - Eccentricity - (au): " + str(e))
  print("i - Inclination - (degrees): " + str(I))
  print("ω - Argument of Periapsis - (degrees): " + str(w))
  print("Ω - Longitude of Ascending Node - (degrees): " + str(ascNode))
  print("M0 - Mean Anomoly at 2022 July 24 6:00 UTC - (degrees): " + str(M0))
  print("M - Mean Anomoly at JD = " + str(inputData['JD'][1]) + " (t2) - (degrees): " + str(M))
  print("E - Eccentric Anomoly at JD = " + str(inputData['JD'][1]) + " (t2) - (degrees): " + str(E))
  print("n - Mean Motion - (deg/d): " + str(n * (180/pi)))
  print("P - Period - (yrs): " + str(P))

  return finalAsteroidPosVector, finalAsteroidVelVector, [a, e, I, ascNode, w, M, M0, E, n, T, P]

In [ ]:
asteroidPositionVector, asteroidVelocityVector, [a, e, I, ascNode, w, M, M0, E, n, T, P] = odCode('/content/Asteroid.txt')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: ComplexWarning: Casting complex values to real discards the imaginary part


r2Mags:  [1.8381491156513388]
Rows:  [1.0629833437257097]
Which roots/rows would you like to utilize? (0 indexed): 0



Iteration Convergance for Asteroid Vector:


What would you like to use for F & G values? (Options: 1st, 3rd, 4th, and Function): Function
Row2Mag Difference (2): -0.0016116762000768503
Row2Mag Difference (3): -0.000496656904011239
Row2Mag Difference (4): -0.0001546370716343226
Row2Mag Difference (5): -4.8171791578566214e-05
Row2Mag Difference (6): -1.5008504190117122e-05
Row2Mag Difference (7): -4.676670240399972e-06
Row2Mag Difference (8): -1.4569318176338442e-06
Row2Mag Difference (9): -4.5391756064994127e-07
Row2Mag Difference (10): -1.418137132169761e-07
Row2Mag Difference (11): -4.421060451065273e-08
Row2Mag Difference (12): -1.3475690341380187e-08
Row2Mag Difference (13): -4.200477965454752e-09
Row2Mag Difference (14): -2.0601305106282553e-09
Row2Mag Difference (15): -6.436842170387536e-10
Row2Mag Difference (16): -2.0066903694271332e-10
Row2Mag Difference (17)